In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("mode.chained_assignment", None)
import warnings
warnings.filterwarnings('ignore')
import tkinter as tk
from tkinter import filedialog
from pprint import pprint
from selectolax.parser import HTMLParser
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
import time
import asyncio
import aiohttp
import sqlite3


In [ ]:
url = "https://civil.bsmi.gov.tw/bsmi_pqn/pqn/uqi1105f.do"
payload = {"state": "queryAll", "progID": "UQI1105F", "progName": "各類貨品檢驗合格名單查詢", "currentPage": 1, "yearMonth": "11110", "kindTypes": "J401"}
headers = headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Accept-Language": "zh-TW,zh;q=0.9,ru;q=0.8,en-US;q=0.7,en;q=0.6,ja;q=0.5",
    "Cache-Control": "max-age=0",
    "Connection": "keep-alive",
    "Content-Type": "application/x-www-form-urlencoded",
    "Cookie": "JSESSIONID=DCNTAqZ2c45Hw2NsEz1fR_U3qKiK1amgTDCcGy14.tp-pqn-01; JSESSIONID=wDhArJ3bibaIKLDENRci8pQwuqUSVVwtUux3sg7e.tp-pqn-01",
    "Origin": "https://civil.bsmi.gov.tw",
    "Referer": "https://civil.bsmi.gov.tw/bsmi_pqn/pqn/uqi1105f.do",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-User": "?1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36",
    "sec-ch-ua": 'Google Chrome";v="107", "Chromium";v="107", "Not=A?Brand";v="24"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"'
}

In [ ]:
res = requests.post(url, headers=headers, params=payload)

In [ ]:
parser = HTMLParser(res.text)

In [ ]:
ttl_pages = int(parser.css_first("div.form-group.row.dataRows > div:nth-child(1) > div > span:nth-child(3)").text().split(",")[1].strip().replace("共","").replace("頁", ""))

In [ ]:
page_list = [i for i in range(1, ttl_pages + 1)]

In [ ]:
def get_data(page):
    test = list()
    url = "https://civil.bsmi.gov.tw/bsmi_pqn/pqn/uqi1105f.do"
    payload = {"state": "queryAll", "progID": "UQI1105F", "progName": "各類貨品檢驗合格名單查詢", "currentPage": page, "yearMonth": "11110", "kindTypes": "J401"}
    headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Accept-Language": "zh-TW,zh;q=0.9,ru;q=0.8,en-US;q=0.7,en;q=0.6,ja;q=0.5",
    "Cache-Control": "max-age=0",
    "Connection": "keep-alive",
    "Content-Type": "application/x-www-form-urlencoded",
    "Cookie": "JSESSIONID=DCNTAqZ2c45Hw2NsEz1fR_U3qKiK1amgTDCcGy14.tp-pqn-01; JSESSIONID=wDhArJ3bibaIKLDENRci8pQwuqUSVVwtUux3sg7e.tp-pqn-01",
    "Origin": "https://civil.bsmi.gov.tw",
    "Referer": "https://civil.bsmi.gov.tw/bsmi_pqn/pqn/uqi1105f.do",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-User": "?1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36",
    "sec-ch-ua": 'Google Chrome";v="107", "Chromium";v="107", "Not=A?Brand";v="24"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"'}
    res = requests.post(url, headers=headers, params=payload).text
    parser = HTMLParser(res)
    for content in parser.css("#listContainer > div.panel-body.listRows > div:nth-child(n+2)"):
                company_content = content.css("div.col-xs-12.col-sm-6.col-md-6.col-lg-4")
                company_name = company_content[0].text().strip() #公司名稱
                other_content = content.css("div.col-xs-12.col-sm-12.col-md-12.col-lg-4")
                id_type = other_content[0].text().strip().replace(" ", "").split("\u3000")[0] #類型
                serial_num = other_content[0].text().strip().replace(" ", "").split("\u3000")[-1]
                num1 = serial_num.split("~")[0] #上排標籤
                num2 = serial_num.split("~")[-1] #下排標籤
                # print(company_content, company_name, other_content, id_type, serial_num, num1, num2)
                line_list = [company_name, id_type, serial_num, num1, num2]
                test.append(line_list)
    return test
                
    time.sleep(0, 1)
    
    

In [ ]:
dfs = list()
with ThreadPoolExecutor(max_workers=20) as executor:
    results = executor.map(get_data, page_list)
for r in results:
    df = pd.DataFrame(r, columns = ["公司名稱", "id_type", "serial_num", "num1", "num2"])
    dfs.append(df)
results = pd.concat(dfs).reset_index(drop=True)
results

In [2]:
class BSMI:
    
    init_url = "https://civil.bsmi.gov.tw/bsmi_pqn/pqn/uqi1105f.do"
    
    payload = {"state": "queryAll", "progID": "UQI5101F", "progName": "檢驗標識-標準檢驗局印製", "currentPage": 1, "yearMonth": "11110", "kindTypes": "J401"}

    headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Accept-Language": "zh-TW,zh;q=0.9,ru;q=0.8,en-US;q=0.7,en;q=0.6,ja;q=0.5",
    "Cache-Control": "max-age=0",
    "Connection": "keep-alive",
    "Content-Type": "application/x-www-form-urlencoded",
    "Cookie": "JSESSIONID=DCNTAqZ2c45Hw2NsEz1fR_U3qKiK1amgTDCcGy14.tp-pqn-01; JSESSIONID=wDhArJ3bibaIKLDENRci8pQwuqUSVVwtUux3sg7e.tp-pqn-01",
    "Origin": "https://civil.bsmi.gov.tw",
    "Referer": "https://civil.bsmi.gov.tw/bsmi_pqn/pqn/uqi1105f.do",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-User": "?1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36",
    "sec-ch-ua": 'Google Chrome";v="107", "Chromium";v="107", "Not=A?Brand";v="24"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"'
}
    
    
    def __init__(self):
        self.months = self.get_months()
        self.df = self.create_df()
    
    def get_months(self):
        response = requests.post(self.init_url, headers=self.headers)
        parser = HTMLParser(response.text)
        available_mon = [i.text() for i in parser.css("#yearMonth > option:nth-child(n+1)")]
        return available_mon
    
    def parse(self, url, payload, header):
        response = requests.post(url, data=payload, headers=header)
        parser = HTMLParser(response.text)
        return parser
    
    def get_ttl_pages(self, parser):
        ttl_page_num = parser.css_first("#form1 > div.form-group.row.dataRows > div:nth-child(1) > div > span:nth-child(3)").text().split(",")[1].strip().replace("共", "").replace("頁", "")
        return ttl_page_num
        
    
    def create_df(self):
        df = pd.DataFrame(columns=["年月", "公司名稱", "類型", "上排標籤", "下排標籤", "品名中文", "品名英文", "項次", "規格", "型號"])
        return df
    
    def extract_data(self, ttl_pages, month):
        df = self.df
        payload = self.payload
        for page in range(1, int(ttl_pages) + 1):
            payload["currentPage"] = page
            parser = self.parse(self.init_url, payload, self.headers)
            for content in parser.css("#listContainer > div.panel-body.listRows > div:nth-child(n+2)"):
                company_content = content.css("div.col-xs-12.col-sm-6.col-md-6.col-lg-4")
                company_name = company_content[0].text().strip() #公司名稱
                other_content = content.css("div.col-xs-12.col-sm-12.col-md-12.col-lg-4")
                id_type = other_content[0].text().strip().replace(" ", "").split("\u3000")[0] #類型
                serial_num = other_content[0].text().strip().replace(" ", "").split("\u3000")[-1]
                num1 = serial_num.split("~")[0] #上排標籤
                num2 = serial_num.split("~")[-1] #下排標籤
                # print(company_name, id_type, serial_num, num1, num2)
                payload_detail = {"state": "queryAll", "progID": "UQI5101F", "progName": "檢驗標識-標準檢驗局印製", "q_markYearPer": id_type, "q_markNo": num1, "currentPage": 1}
                check_url = "https://civil.bsmi.gov.tw/bsmi_pqn/pqn/uqi5101f.do"
                parser_detail = self.parse(check_url, payload_detail, self.headers)
                cn_name = []
                en_name = []
                item = []
                size = []
                model = []
                for detail in parser_detail.css("#listDetailContainer > div > div"):
                    keys = detail.css("label.control-label.col-xs-4.col-sm-3.col-md-3.col-lg-3")[0].text().strip()
                    values = detail.css("div.col-xs-8.col-sm-9.col-md-9.col-lg-9")[0].text().strip()
                    if "品名中文" in keys:
                        cn_name.append(values)
                    elif "品名英文" in keys:
                        en_name.append(values)
                    elif "項次" in keys:
                        item.append(values)
                    elif "規格" in keys:
                        size.append(values)
                    else:
                        model.append(values)  
                s = pd.Series([month, company_name, id_type, num1, num2, cn_name, en_name, item, size, model], index=["年月", "公司名稱", "類型", "上排標籤", "下排標籤", "品名中文", "品名英文", "項次", "規格", "型號"])
                # print(s)
                df = df.append(s, ignore_index=True)
                # print(df)
        return df
    
    
    def get_companies_info(self, month=None):
        start_time = datetime.now()
        print("開始爬取數據")
        payload = self.payload
        if month == None:
            dfs = []
            for month in self.months:
                payload["yearMonth"] = month
                # print(payload)
                parser = self.parse(self.init_url, payload, self.headers)
                ttl_pages = self.get_ttl_pages(parser)
                year_month = month
                # print(ttl_pages)
                df = self.extract_data(ttl_pages, year_month)
                dfs.append(df)
            df = pd.concat(dfs).reset_index(drop=True)
            df = self.get_result(df)
            end_time = datetime.now()
            print(end_time - start_time)
            return df
        else:
            payload["yearMonth"] = month
            # print(payload)
            parser = self.parse(self.init_url, payload, self.headers)
            ttl_pages = self.get_ttl_pages(parser)
            year_month = month
            df = self.extract_data(ttl_pages, year_month)
            end_time = datetime.now()
            df = self.get_result(df)
            print(end_time - start_time)
            return df
        
        
        
    def get_result(self, df):
        df = df.explode(["品名中文", "品名英文", "項次", "規格", "型號"], ignore_index=True)
        df = df.astype({"上排標籤": int, "下排標籤": int})
        df.insert(5, "批次數量", df.apply(lambda x: x["下排標籤"] - x["上排標籤"], axis=1))
        return df
        
            
                    
                
                
            
            
        
    

In [3]:
df = BSMI()

In [4]:
result = df.get_companies_info()

開始爬取數據


AttributeError: 'DataFrame' object has no attribute 'self'

In [ ]:
url = "https://civil.bsmi.gov.tw/bsmi_pqn/pqn/uqi1105f.do"
test = requests.post(url, data={"state":"queryAll", "progID":"UQI1105F", "progName":"各類貨品檢驗合格名單查詢", "yearMonth": "11109", "kindTypes": "J401", "currentPage": 1})
test = BeautifulSoup(test.text)
pages = int(test.find("div", class_="col-xs-12 col-sm-6 col-md-6 col-lg-6").getText().strip().split(",")[1].replace("共", ",").replace("頁", ",").split(",")[-2])

In [ ]:
columns = ["公司", "類型", "證號"]
df = pd.DataFrame(columns=columns)
official = requests.post(url, data={"state":"queryAll", "progID":"UQI1105F", "progName":"各類貨品檢驗合格名單查詢", "yearMonth": "11109", "kindTypes": "J401", "currentPage": 1})
res = BeautifulSoup(official.text)
for i in res.select("#listContainer > div.panel-body.listRows > div.row"):
    # pprint(i)
    y = i.findAll("div", class_="col-xs-12 col-sm-6 col-md-6 col-lg-4")
    serial = i.find("div", class_="col-xs-12 col-sm-12 col-md-12 col-lg-4").get_text()
    data = [y[0].get_text().strip(), y[-1].get_text().strip(), serial.strip()]
    s = pd.Series(data, index=["公司", "類型", "證號"])
    df = df.append(s, ignore_index=True)

In [ ]:
df = df[~(df["公司"] == "廠商名稱")]
df["證號"] = df["證號"].map(lambda x: x.replace(" ", ""))
df["證號長度"] = df["證號"].map(lambda x: len(x))
df = df[df["證號長度"] != 0]
df["證號"] = df["證號"].str.strip()
serial_numbers = df["證號"].str.split(expand=True)
volume_cal = serial_numbers[1].str.split("~", expand=True)
df["上排標籤"] = serial_numbers[0]
df["起始序號"] = volume_cal[0]
df["結束序號"] = volume_cal[1]
df["數量"] = df["結束序號"].astype(int) - df["起始序號"].astype(int)
df["日期"] = "11109"
df = df[["日期", "公司", "類型", "上排標籤", "起始序號", "結束序號", "數量"]]
df["批次"] = df.groupby(["公司"]).cumcount() + 1

In [ ]:
df.to_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\標準檢驗局\測試資料.xlsx", index=False)

In [ ]:
requests.post("https://civil.bsmi.gov.tw/bsmi_pqn/ajax/jsonGetBrowseByProgID.jsp", headers=headers, params={"q": "UQI1105F"}).text()

In [9]:

conn = sqlite3.connect(r"C:\Users\kc.hsu\Python workspace\BS_Spider\web_scraping\web_scraping\scraping_df")
df = pd.read_sql("select * from bsmi", con=conn)

In [12]:
df.shape

(172148, 11)

In [14]:
result = df.drop_duplicates()

In [16]:
cursor = conn.cursor()

In [17]:
cursor.execute("delete from bsmi")

In [19]:
conn.commit()

In [22]:
for index, row in result.iterrows():
    cursor.execute('insert into bsmi (month, company_name, tyre_type, category, upper_tag, lower_tag, cn_product_name, en_product_name, items, size, model) values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', tuple(row))
conn.commit()

In [24]:
cursor.close()

In [25]:
result.to_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\標準檢驗局\20230221_商檢局資料.xlsx", index=False)